# Яндекс Практикум Geonames

**Заказчик**
Карьерный центр Яндекс Практикум

**Описание проекта**

***Цель***:
- Сопоставление произвольных гео названий с унифицированными именами geonames для внутреннего использования Карьерным центром

**Задачи:**

- Создать решение для подбора наиболее подходящих названий с geonames. Например Ереван -> Yerevan
- На примере РФ и стран наиболее популярных для релокации - Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия. Города с населением от 15000 человек (с возможностью масштабирования на сервере заказчика)
- Возвращаемые поля geonameid, name, region, country, cosine similarity
- формат данных на выходе: список словарей, например [{dict_1}, {dict_2}, …. {dict_n}] где словарь - одна запись с указанными полями

***Задачи опционально:***

- возможность настройки количества выдачи подходящих названий (например в параметрах метода)
- коррекция ошибок и опечаток. Например Моченгорск -> Monchegorsk
- хранение в PostgreSQL данных geonames
- хранение векторизованных промежуточных данных в PostgreSQL
- предусмотреть методы для настройки подключения к БД
- предусмотреть метод для инициализации класса (первичная векторизация geonames)
- предусмотреть методы для добавления векторов новых гео названий

## План работы

Проект содержит три основные части:

1. Работа с исходными данными и создание датасетов.
2. Работа с базой данный: создание, наполнение, извлечение.
3. Поиск городов.

Ниже чуть более подробно по каждому пункту. 

### Работа с исходными данными и создание датасетов

По условию задачи необходимо на примере РФ и стран наиболее популярных для релокации - Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия. Города с населением от 15000 человек. 
Датасет с городами будет создан из городов с налеселением от 500 человек, в даньшейшем численность населения можно будет выбрать в `SQL` запросе к `БД`.

Будут созданы следующие датасеты:

- `cities`: информация о городах,
- `countries`: информация о странах;
- `admin_codes`: информация об областях ;
- `embeddings`: векторные представления названий городов.

Будет создан класс для создания датасетов, а также несколько функция для обработки.

### Работа с базой данный: создание, наполнение, извлечение

После создания датасетов, можно реализовать следующую схему для `БД`.

<img src="geonames_er_diagram.png" alt="Описание">

Работа с `БД` будет проходить в следующих этапах:

- создание `БД` (отдельный класс),
- создание таблиц `БД` - создание на основе `declarative_base`,
- заполнение таблиц `БД` (отдельный класс для заполнения и извлечения данных из таблиц) с предусмотренной возможностью снятия ограничений по внешним ключам и перезаписью или добавлением данных,
- извлечение данных из `БД` при помощи `SQL` запроса.

### Поиск городов

Будет реализован отдельный класс для поиска городов, предусматривающий следующее:

- коррекцию опечаток,
- расширенную коррекцию опечаток или сокращений при помощи поска по аьтернативным названиям городов,
- возможность выбора вывода информации в виде таблицы или словаря,
- возможносьть сохранения словаря в `JSON` файл. 

## Библиотеки и переменные

**Библиотеки необходимые в проекте**

In [1]:
# базовые импорты
import gc
import json
import os
import numpy as np
import pandas as pd
from pprint import pprint

# импорты для работы с БД
import psycopg2
from psycopg2.extensions import AsIs, register_adapter
from sqlalchemy import (
    ARRAY,
    REAL,
    Column,
    Float,
    ForeignKey,
    Integer,
    String,
    create_engine,
    text,
)
from sqlalchemy.orm import declarative_base, relationship

# импорты для работы векторами
import torch
from sentence_transformers import SentenceTransformer, util

# импорты для коррекции ошибок
from fuzzywuzzy import process
from transliterate import translit
from yaspeller import check

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 100)

RANDOM = 12345
torch.manual_seed(RANDOM)
np.random.seed(RANDOM)

**Переменные для датасетов**

In [2]:
# рабочая директория проекта
WORK_DIR = os.path.abspath(os.curdir)
# директория c txt исходниками проекта
SRC_DIR = os.path.join(WORK_DIR, 'source')
# директория для сохранения датасетов
DATA_DIR = os.path.join(WORK_DIR, 'datasets')
# директория для сохранения json файлов с результатом
OUT_DIR = os.path.join(WORK_DIR, 'output')

# Нижеперечисленные переменные будут использованы при загрузке в методе load_dataset класса DatasetLoader
CITY_FILE = "cities500.txt"  # файл с городами
COUNTRY_FILE = "countryInfo.txt"  # файл со странами
ADMIN_CODE_FILE = "admin1CodesASCII.txt"  # файл с областями
# названия столбцов для датасета с городами
CITY_COLS = [
    "city_geoname_id",
    "name",
    "asciiname",
    "alternatenames",
    "latitude",
    "longitude",
    "feature_class",
    "feature_code",
    "country_code_iso",
    "cc2",
    "admin_1_code",
    "admin_2_code",
    "admin_3_code",
    "admin_4_code",
    "population",
    "elevation",
    "dem",
    "timezone",
    "modification_date",
]
# названия столбцов которые будут выведены в датасет с городами
USE_CITY_COLS = [
    "city_geoname_id",
    "name",
    "asciiname",
    "alternatenames",
    "latitude",
    "longitude",
    "feature_class",
    "feature_code",
    "country_code_iso",
    "admin_1_code",
    "population",
    "timezone",
]
# типы данных для некоторых столбцов, заданные по умолчанию при загрузке
COL_TYPES = {"country_code_iso": str, "admin_1_code": str}
# названия столбцов для датасета со странами
COUNTRY_COLS = [
    "iso",
    "iso_3",
    "iso_numeric",
    "fips",
    "country",
    "capital",
    "area_in_sq_km",
    "population",
    "continent",
    "tld",
    "currency_code",
    "currency_name",
    "phone",
    "postal_code_format",
    "postal_code_regex",
    "languages",
    "county_geoname_id",
    "neighbours",
    "equivalent_fips_code",
]
# названия столбцов которые будут выведены в датасет со странами
USE_COUNTRY_COLS = [
    "iso",
    "iso_3",
    "country",
    "capital",
    "area_in_sq_km",
    "population",
    "continent",
    "tld",
    "currency_code",
    "currency_name",
    "phone",
    "languages",
]
# названия столбцов для датасета собластями
ADMIN_COLS = ["admin_code", "name", "name_ascii", "geoname_id"]
# названия столбцов которые будут выведены в датасет с областями
USE_ADMIN_COLS = ["admin_code", "name", "name_ascii"]

**Переменные для БД**

In [3]:
# словарь с конфигурацией для подключения к БД
db_config = {
    "user": "andreivk",  # имя пользователя
    "pwd": "andreivk_1980",  # пароль
    "host": "localhost",  # хост
    "port": 5432,  # порт подключения
    "db": "geonames",  # название базы данных
    "default_db": "postgres",  # дефолтная база данных для созднания новой БД
}
# коннекшн строка для инициализации подключения к БД geonames
CONN_STR_GEONAMES = "postgresql://{}:{}@{}:{}/{}".format(
    db_config["user"],
    db_config["pwd"],
    db_config["host"],
    db_config["port"],
    db_config["db"],
)
# коннекшн строка для инициализации подключения к дефолтной БД postgres для создания БД geonames
CONN_STR_DEFAULT = "postgresql://{}:{}@{}:{}/{}".format(
    db_config["user"],
    db_config["pwd"],
    db_config["host"],
    db_config["port"],
    db_config["default_db"],
)

# список стран для поиска
# по умолчанию взяты Россия и Казахстан
# но можно составить список с изначальным условием, правильное написание стран в файле countryInfo.txt
# ["Russia", "Kazakhstan", "Belarus", "Armenia", "Azerbaijan", "Georgia", "Serbia", Turkey"]
COUNTRIES_LST = ["Russia", "Kazakhstan"]
# численность населения
POPULATION = 15000
# SQL - запрос к БД
QUERY = """
  SELECT ci.city_geoname_id as geoname_id,
        ci.name,
        ci.alternatenames,
        ad.name as oblast,
        co.country,
        co.capital,
        co.currency_name,
        ci.timezone,
        ci.latitude,
        ci.longitude,
        em.embeddings
  FROM city AS ci
  JOIN country AS co ON ci.country_code_iso = co.iso
  JOIN embeddings AS em ON em.name = ci.name
  JOIN admincode AS ad ON ad.admin_code = ci.admin_code 
  WHERE co.country IN ({}) AND ci.population >= {}
  ORDER BY ci.name ASC;
         """

**Переменные для моделирования эмбеддингов и вывода результата**

In [4]:
# проверяем наличие gpu
DEVICE = torch.device(
    "cuda:0") if torch.cuda.is_available() else torch.device("cpu")
# имя модели sentence-transformers
MODEL_ID = "sentence-transformers/LaBSE"
# список выводимых столбцов для результирующей таблицы.
COLS_OUTPUT = [
    "geoname_id",
    "name",
    "oblast",
    "country",
    "capital",
    "currency_name",
    "timezone",
    "latitude",
    "longitude",
]

## Создание датасетов

Для датасетов будет создан класс загрузчик `DatasetLoader` и функции для обработки: `reduce_mem_usage`, `preprocess_data`, `remove_difference`.

Более детально описано ниже.

**Функция для приведения типов данных к оптимальным согласно разбросу значений** 

In [5]:
def reduce_mem_usage(df):
    """
    Функция перебирает все столбцы датафрейма и изменяеет тип данных, чтобы
    уменьшить использование памяти
    Параметр:
            df (pd.Dataframe): исходный датасет.
    Возвращаемое значение:
            df (pd.Dataframe): оптимизированный датасет.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(
        "Память занимаемая датасетом в ОП до обработки: {:.4f} MB".format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if (
            col_type != object
            and col_type.name != "category"
            and "datetime" not in col_type.name
        ):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif "datetime" not in col_type.name:
            df[col] = df[col].astype("object")

    end_mem = df.memory_usage().sum() / 1024**2
    print(
        "Память занимаемая датасетом в ОП после обработки: {:.4f} MB".format(end_mem))
    print("Экономия {:.2f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df

**Класс `DatasetLoader` для загрузки и сохранения датасетов**

Данный класс будет создавать датасеты и сохранять их в файлы.

Методы класса:
- load_dataset: метод загрузчик датасета из файла txt или csv,
- load_city_embeddings: метод для создания датасета с векторами слов,
- save_dataset_to_file: сохраняет датасет в файл,
- is_accessible (staticmethod): статический метод для проверки доступности файлов в режиме чтения.

В классе предусмотрены след. возможности:

- метод `load_dataset`.
    - передавать названия столбцов для датасета из переменных `CITY_COLS`, `COUNTRY_COLS`, `ADMIN_COLS` объявленных ранее,
    - передавать названия столбцов для датасета, которые будут отображены из переменных `USE_CITY_COLS`, `USE_COUNTRY_COLS`, `USE_ADMIN_COLS` объявленных ранее,
    - задавать сразу типы данных для некоторых столбцов, например для датасета с городами - `COL_TYPES`.
-  метод `save_dataset_to_file`, сохраняет датасеты в `pickle` файлы c `zip` компрессией.

In [6]:
class DatasetLoader:
    """
    Класс DatasetLoader.
    Класс загружает датасет из txt или csv файлов и создает датасет с векторными
    представлениями слов из созданных датасетов.

    Методы класса:
        load_dataset: метод загрузчик датасета из файла txt или csv,
        load_city_embeddings: метод для создания датасета с векторами слов,
        save_dataset_to_file: сохраняет датасет в файл,
        is_accessible (staticmethod): статический метод для проверки доступности файлов в режиме чтения.
    """

    def __init__(self, work_dir=None):
        """
        Инициализация объекта класса DatasetLoader.
        Параметр:
            work_dir (str): рабочая директория с файлами для датасета, по умолчанию None.
        """
        self.work_dir = work_dir

    @staticmethod
    def is_accessible(file, work_dir, mode="r"):
        """
        Статический метод is_accessible класса DatasetLoader.
        Проверка, является ли файл в рабочей директории
        доступным для работы в предоставленом `mode` формате.
        Параметры:
            file (str): файл с данными для датасета,
            work_dir (str): рабочая директория с файлами для датасета,
            mode (str): режим доступа к файлу, по умолчанию равно 'r' - чтение.
        Возвращаемое значение.
            Boolean: True или False.
        """
        try:
            f = open(os.path.join(work_dir, file), mode)
            f.close()
        except IOError:
            return False
        return True

    def load_dataset(self, file=None, df_cols=None, use_cols=None, col_types=None):
        """
        Метод load_dataset для загрузки датасета из файла txt или csv.
        Параметры:
            file (str): файл с данными для датасета, по умолчанию None,
            df_cols (list): полный список с названиями колонок в датасете, по умолчанию равно None,
            use_cols (list): список с названиями колонок, которые будут отображены в датасете,
                             по умолчанию равно None,
            col_types (dict): словарь с колонками и типами, например {"col_name": str}, для
                              изначального назначения типов данных столбцу или столбцам,
                              по умолчанию равно None.
        Возвращаемое значение:
            dataset (pd. Dataframe): созданный датафрейм Pandas.
        """
        # проверка типа переменной file на строковое значение
        if not isinstance(file, str):
            raise TypeError(
                f"Не соответствует тип переменной file, должен быть тип str. Датасет не будет создан."
            )
        # проверка переменной file на ненулевое значение
        if len(file) == 0:
            raise ValueError(
                f"file не должен быть пустой строкой, списком. Датасет не будет создан."
            )
        # вызов статического метода is_accessible для проверки файла на доступность
        flag = DatasetLoader.is_accessible(file, self.work_dir)
        # если True, то создаем датасет
        if flag:
            print(f"Создаем датасет из файла {file}...")
            # стандарнтый pd.read_csv метод пандас для загрузки датасета
            dataset = pd.read_csv(
                os.path.join(self.work_dir, file),
                header=None,
                names=df_cols,
                usecols=use_cols,
                dtype=col_types,
                delimiter="\t",
                low_memory=False,
            )
            print(f"Датасет создан!")
            # возвращаемый датасет
            return dataset
        #
        else:
            # в случае отсутствия файла возврат ValueError
            raise ValueError(
                f"Файл {file} на найден в директории {self.work_dir}.")

    def load_city_embeddings(
            self,
            device="cpu",
            model_id=None,
            batch_size=8,
            id_emb_col=None,
    ):
        """
        Метод load_city_embeddings для создания датасета и векторов слов из колонки датасета.
        Параметры:
            device (str): акселератор для создания векторов CPU или GPU, по умолчанию равно 'cpu',
            model_id (str): имя модели для векторизации, по умолчанию равно None,
            batch_size (int): размер батча для создания векторов слов, по умолчанию равно 8,
            id_emb_col (pd.Series или list): столбец с текстом для векторизации, по умолчанию равно None,
            save_to_file (bool): флаг, указывающий, нужно ли сохранять датасет в файл, по умолчанию равно False,
            file_name (str): имя файла для сохранения датасета, по умолчанию равно 'embeddings',
            dir_to_save (str): директория для сохранения датасета.
        Возвращаемое значение:
            dataset (pd.Dataframe): созданный датафрейм Pandas.
        """
        # проверка типа переменной id_emb_col на тип list или pd.Series
        if not isinstance(id_emb_col, (list, pd.Series)):
            raise TypeError(
                f"Не соответствует тип переменной id_emb_col, должен быть тип list или pd.Series. Датасет не будет "
                f"создан."
            )
        # проверка переменной id_emb_col на ненулевое значение
        if len(id_emb_col) == 0:
            raise ValueError(
                f"Количество элементов в id_emb_col не должно быть нулевым. Датасет не будет создан."
            )
        # если проверки пройдены создаём датасет
        else:
            # создаём пустой датасет
            dataset = pd.DataFrame()
            # в список берем только уникальные названия городов
            id_emb_col = list(set(id_emb_col))
            # создаем столбец с названиями городов
            dataset["name"] = id_emb_col
            # загрузка модели для создания векторов
            print(f"Загружаем модель для создания эмбеддингов ...")
            model = SentenceTransformer(model_id)
            # создание векторов
            print(
                f"Создание эмбеддингов...  Размер батча --> {batch_size}, CPU или GPU --> {device} ..."
            )
            embeddings = model.encode(
                id_emb_col, show_progress_bar=True, device=device, batch_size=batch_size
            )
            # добавление в датасет столбца с векторами слов
            dataset["embeddings"] = list(embeddings)
            print(f"Датасет создан!")
            # удаление переменных и очистка памяти CUDA
            del model
            del embeddings
            del id_emb_col
            gc.collect()
            torch.cuda.empty_cache()
            torch.cuda.reset_peak_memory_stats()
            # возвращаемый датасет
            return dataset

    def save_dataset_to_file(self, dataset=None, file_name=None, dir_to_save=None):
        """
        Метод save_dataset_to_file для сохранения датасета в файл на диске.
        Параметры:
            dataset (pd.Dataframe): датафрейм Pandas для сохранения на диск.
            file_name (str): имя файла для сохранения датасета, по умолчанию равно'embeddings',
            dir_to_save (str): директория для сохранения датасета.
        Возвращаемое значение:
            Остсутствует.
        """
        # сохраняем датасет в файл методом to_pickle с zip компрессией для оптимизации места на диске
        print(f"Сохраняем датасет в файл {file_name} ...")
        dataset.to_pickle(os.path.join(
            dir_to_save, file_name), compression="zip")

**Функция обработки датасетов**

Т.к. датасеты необходимо немного обработать, данная функция будет полезна для обработки.

Этапы:
- Датасет с городами `cities`.
    - в столбце `alternatenames` имена идут через запятую без пробелов - добавляем пробел;
    - удаляем строки с попусками в стобцах `admin_1_code`, `name`, `country_code_iso`;
    - создаём новый столбец `admin_code` в виде суммы через точку столбцов `country_code_iso` и `admin_1_code`;
    - удаляем столбец `admin_1_code`;
    - заполняем пропуски в столбце `alternatenames` на основе столбца `names`;
    - заполняем пропуски в столбце `asciiname` на основе столбца `names`.
- Датасет со странами `countries`.
    - в столбце `languages` имена идут через запятую без пробелов - добавляем пробел;
    - при загрузке стран `Pandas` почему-то определяет страну `Namibia`, код `iso`: `NA`, как `NaN`, поэтому нужно вернуть код `NA`

In [7]:
def preprocess_data(dataset=None, city_or_country=None):
    """
    Функция предобработки датасетов с городами или странами.
    Параметры:
            dataset (pd.Dataframe): исходный датасет, по умолчанию равно None,
            city_or_country (str): выбора датасета с городами или странами,
                                   по умолчанию равно None.
                                   Если параметр = "city", то обрабатывается датасет с городами,
                                   если параметр = "country", то обрабатывается датасет с городами,
                                   в противном случае выводистя сообщение о невозможности обработки.
    Возвращаемое значение:
            dataset (pd.Dataframe): обработанный датасет.
    """
    # если датасет с городами
    if city_or_country == "city":
        dataset = dataset.copy()
        # добавляем пробел в столбце alternatenames
        dataset["alternatenames"] = dataset["alternatenames"].str.replace(
            ",", ", ")
        # удаляем строки с попусками в стобцах admin_1_code, name, country_code_iso
        dataset = dataset.dropna(
            subset=["admin_1_code", "name", "country_code_iso"])
        # создаём новый столбец admin_code в виде суммы через точку столбцов country_code_iso и admin_1_code
        dataset.loc[:, "admin_code"] = (
            dataset["country_code_iso"] + "." + dataset["admin_1_code"]
        )
        # удаляем столбец admin_1_code
        dataset = dataset.drop(["admin_1_code"], axis=1)
        # заполняем пропуски в столбце asciiname на основе столбца names
        dataset.loc[dataset["alternatenames"].isna(), "alternatenames"] = dataset[
            "name"
        ]
        # возврат iso кода NA Намибии
        dataset.loc[dataset["asciiname"].isna(), "asciiname"] = dataset["name"]
        print("Датасет обработан!")
    # если датасет со странами
    elif city_or_country == "country":
        dataset = dataset.copy()
        # добавляем пробел в столбце languages
        dataset["languages"] = dataset["languages"].str.replace(",", ", ")
        #
        dataset.loc[dataset["country"] == "Namibia", "iso"] = "NA"
        print("Датасет обработан!")
    # в противном случае
    else:
        print(
            """
              Не выбран датасет для обработки.
              Вы вабрали значение параметра city_or_country={}.
              Нужно указать правильное значение параметра city_or_country:
              - для городов - 'city';
              - для стран - 'country'.
              Параметр д.б. строкой.
              """.format(
                city_or_country
            )
        )
    # возврат результата обработки
    return dataset

**Создаем экземпляр класса `DatasetLoader`**

In [8]:
# Создаем экземпляр класса
loader = DatasetLoader(work_dir=SRC_DIR)

***Загружаем данные о городах***

In [9]:
# Загружаем данные о городах через функцию reduce_mem_usage
cities = reduce_mem_usage(
    loader.load_dataset(
        file=CITY_FILE, df_cols=CITY_COLS, use_cols=USE_CITY_COLS, col_types=COL_TYPES
    )
)

Создаем датасет из файла cities500.txt...
Датасет создан!
Память занимаемая датасетом в ОП до обработки: 18.3718 MB
Память занимаемая датасетом в ОП после обработки: 14.5444 MB
Экономия 20.83%


In [10]:
cities.sample(3)

,city_geoname_id,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code_iso,admin_1_code,population,timezone
163620,518621,Novomalorossiyskaya,Novomalorossiyskaya,"Grazhdanskaya,Novomalorossijskaja,Novomalorossiskaya,Novomalorossiyskaya,Novomyshastovskaya,Ново...",45.625000,39.906250,P,PPL,RU,38,5867,Europe/Moscow
83545,260144,Chálki,Chalki,"Chalki,Chálki,Khalki,Khálki,Maimouli,Maïmoúli,Χάλκη",39.562500,22.546875,P,PPL,GR,ESYE14,1837,Europe/Athens
130810,4026952,Pantla,Pantla,NaN,17.734375,-101.625000,P,PPL,MX,12,3917,America/Mexico_City


Как видно из вывода в дальнейшем нужно применить функцию `preprocess_data` для:

- добавления пробелов столбце `alternatenames`;
- удаления строк с попусками в стобцах `admin_1_code`, `name`, `country_code_iso`;
- создания нового столбца `admin_code` в виде суммы через точку столбцов `country_code_iso` и `admin_1_code`;
- удаления столбца `admin_1_code`;
- заполнения пропусков в столбце `alternatenames` на основе столбца `names`;
- заполнения пропусков в столбце `asciiname` на основе столбца `names`.

***Обработка датасета `cities`***

In [11]:
cities = preprocess_data(dataset=cities, city_or_country="city")

Датасет обработан!


***Загружаем данные о странах***

In [12]:
# Загружаем данные о странах через функцию reduce_mem_usage
countries = reduce_mem_usage(
    loader.load_dataset(
        file=COUNTRY_FILE, df_cols=COUNTRY_COLS, use_cols=USE_COUNTRY_COLS
    )
)

Создаем датасет из файла countryInfo.txt...
Датасет создан!
Память занимаемая датасетом в ОП до обработки: 0.0232 MB
Память занимаемая датасетом в ОП после обработки: 0.0213 MB
Экономия 8.29%


In [13]:
countries.loc[countries["country"] == "Namibia"]

,iso,iso_3,country,capital,area_in_sq_km,population,continent,tld,currency_code,currency_name,phone,languages
160,NaN,NAM,Namibia,Windhoek,825418.0,2448255,AF,.na,NAD,Dollar,264,"en-NA,af,de,hz,naq"


Как видно из вывода в дальнейшем нужно применить функцию `preprocess_data` для:

- добавления пробелов в столбце `languages`;
- возврата `Namibia`, кода `iso`: `NA`.

***Обработка датасета `countries`***

In [14]:
countries = preprocess_data(dataset=countries, city_or_country="country")

Датасет обработан!


***Загружаем данные о об областях***

In [15]:
# Загружаем данные об областях через функцию reduce_mem_usage
admin_codes = reduce_mem_usage(
    loader.load_dataset(
        file=ADMIN_CODE_FILE, df_cols=ADMIN_COLS, use_cols=USE_ADMIN_COLS
    )
)

Создаем датасет из файла admin1CodesASCII.txt...
Датасет создан!
Память занимаемая датасетом в ОП до обработки: 0.0890 MB
Память занимаемая датасетом в ОП после обработки: 0.0890 MB
Экономия 0.00%


**Функция устранения разности в значениях датасетов `cities` и `admin_codes`**

В результате загрузки датасетов может образоваться следующая ситуация: в датасете `cities` для какого-то из городов будет доступен код `admin_code`, например `FW.00`, 
но этого кода нету в таблице `admin_codes`. Для свзяи между таблицами в `БД` наличие этих кодов необходимо в обеих таблицах. Поэтому придется в таблицу `admin_codes`,
добавлять коды за счет разницы в множествах кодов обеих таблиц, и заполнять пропуски и именах областей как `No admin`.

In [16]:
def remove_difference(cities=None, admin_codes=None):
    """
    Функция для устранения разницы в значениях между полями:
     - admin_code в датасете с городами,
     — admin_code в датасете с областями.
    Т.к. в таблице с городами admin_code это внешний ключ, ссылающийся
    на первичный ключ admin_code в таблице с областями, значения ключа, которые
    есть в таблице с городами, должны обязательно быть в таблице с областями.
    Параметры:
            cities (pd.Dataframe): датасет с городами, по умолчанию равно None,
            admin_codes (pd.Dataframe): датасет с областями, по умолчанию равно None
    Возвращаемое значение:
            admin_codes (pd.Dataframe): обработанный датасет с областями.
    """
    # делаем список из значений, которые есть в cities, но нет в admin_codes
    # для этого находим разность множеств cities["admin_code"] и admin_codes["admin_code"]
    difference = list(set(cities["admin_code"]) -
                      set(admin_codes["admin_code"]))
    # создаём новый датафрейм с один столбцом admin_code и значениями из списка difference
    new_rows = pd.DataFrame({"admin_code": difference})
    # добавление значений из списка в столбец DataFrame
    admin_codes = pd.concat([admin_codes, new_rows], ignore_index=True)
    # заполняем пропуски в столбцах "name", "name_ascii" значение No admin, т.к неизвестно к какой области это
    # относится.
    admin_codes.loc[
        admin_codes["admin_code"].isin(difference), ["name", "name_ascii"]
    ] = "No admin"
    return admin_codes

***Обработка датасета `admin_codes`***

In [17]:
admin_codes = remove_difference(cities=cities, admin_codes=admin_codes)

***Загружаем данные с эмбеддингами названий городов*** 

In [18]:
# Загружаем данные с эмбеддингами
embeddings = loader.load_city_embeddings(
    device=DEVICE, model_id=MODEL_ID, id_emb_col=cities["name"]
)

Загружаем модель для создания эмбеддингов ...
Создание эмбеддингов...  Размер батча --> 8, CPU или GPU --> cuda:0 ...


Batches:   0%|          | 0/21163 [00:00<?, ?it/s]

Датасет создан!


***Информация по датасетам*** 

In [19]:
for dataset in [cities, countries, admin_codes, embeddings]:
    print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
Index: 200593 entries, 0 to 200667
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   city_geoname_id   200593 non-null  int32  
 1   name              200593 non-null  object 
 2   asciiname         200593 non-null  object 
 3   alternatenames    200593 non-null  object 
 4   latitude          200593 non-null  float16
 5   longitude         200593 non-null  float16
 6   feature_class     200593 non-null  object 
 7   feature_code      200593 non-null  object 
 8   country_code_iso  200593 non-null  object 
 9   population        200593 non-null  int32  
 10  timezone          200593 non-null  object 
 11  admin_code        200593 non-null  object 
dtypes: float16(2), int32(2), object(8)
memory usage: 20.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  

Где-то есть пропуски, но это не критично - оставляем как есть.

***Сохраняем датасеты в файлы на диск***

In [20]:
for dataset, file_name in zip([cities, countries, admin_codes, embeddings], ["cities", "countries", "admin_codes", "embeddings"]):
    loader.save_dataset_to_file(
        dataset=dataset, file_name=file_name, dir_to_save=DATA_DIR)

Сохраняем датасет в файл cities ...
Сохраняем датасет в файл countries ...
Сохраняем датасет в файл admin_codes ...
Сохраняем датасет в файл embeddings ...


## Создание БД `geonames`

В принципе базу данных можно создать либо через терминал, либо через админку, например `dbeaver`. Но для стройности проекта, я реализовал создание базы через класс `CreateDatabase`.

При установке `postgres` создается дефолтная база `postgres`. Чтобы создать новую базы, а она будет называться `geonames`, можно подключиться к дефолтной базе, через ранее объявленную 
строку подключения `CONN_STR_DEFAULT` и стандарнным `SQL`-запросом создать новую базу данных.

In [21]:
class CreateDatabase:
    """
    Класс CreateDatabase для создания базы данных.
    """

    def __init__(self, conn_str=None):
        """
        Инициализация объекта класса CreateDatabase.

        Параметры:
            conn_str (str): строка подключения к базе данных,
                            по умолчанию равно None.
        """
        self.conn_str = conn_str

    def create_db(self, db_name=None):
        """
        Метод create_db.
        Создание базы данных.

        Параметры:
            db_name (str, optional): имя создаваемой базы данных,
                                     по умолчанию равно None.
        """
        # создание подключения к базе данных
        engine = create_engine(self.conn_str)
        conn = engine.connect()

        # выполнение SQL-запроса для создания базы данных
        conn.execution_options(isolation_level="AUTOCOMMIT").execute(
            text(f"CREATE DATABASE {db_name}")
        )
        conn.close()  # Закрытие соединения

        # вывод сообщения об успешном создании базы данных
        print(f"База данных {db_name} создана!")

**Создаем экземпляр класса `CreateDatabase`**

In [22]:
database = CreateDatabase(conn_str=CONN_STR_DEFAULT)

**Создаем БД `geonames`**

In [23]:
database.create_db(db_name="geonames")

База данных geonames создана!


## Создание таблиц `admincode`, `embeddings`, `country`, `city` в БД `geonames`

Таблицы будут созданы при помощи `declarative_base()`.

`declarative_base()` - это класс, предоставляемый библиотекой `SQLAlchemy`, который является частью её объектно-реляционной модели (`ORM`). Этот класс позволяет создавать классы, представляющие таблицы в базе данных, с использованием привычного синтаксиса `Python`, и позволяет использовать объекты этих классов для выполнения запросов к базе данных.

Когда используется `declarative_base()`, сначала создаётся экземпляр этого класса, который будет являться базовым классом для всех классов, представляющих таблицы в базе данных. Далее, определяются классы, используя средства, предоставляемые библиотекой `SQLAlchemy`, такие как декларативное объявление (`declarative declaration`). При объявлении этих классов, указывается, какие они будут иметь атрибуты, соответствующие столбцам таблицы.

**Создаем экземпляр класса `declarative_base()`**

In [24]:
Base = declarative_base()

Соласно схеме данных будут созданы след. таблицы.

### Класс `Vectors` для таблицы `embeddings`

Таблица `embeddings` содержит следующие поля:

- `name`: `Primary Key` - названия городов;
- `embeddings` - векторное представление названий городов.

In [25]:
class Vectors(Base):
    """
    Класс Vectors для хранения векторных представлений географических названий.
    """
    # имя таблицы
    __tablename__ = "embeddings"
    # комментарий с описанием таблицы
    __table_args__ = {
        "comment": "Таблица с векторными представлениями географических названий."
    }
    # задаем в переменные параметры столбцов в таблице БД, имя переменной является именем столбца
    name = Column(
        String,
        nullable=False,
        unique=True,
        primary_key=True,
        comment="Наименование географического объекта"
    )
    embeddings = Column(
        ARRAY(REAL),
        comment="Векторные представления географического объекта"
    )

    def __repr__(self):
        """
        Метод __repr__.
        Возвращает строковое представление объекта.

        Возвращаемое значение:
            Строка (str): строковое представление объекта с названиями столбцов в таблице.
                          Столбцы:
                            - name,
                            - embeddings.
        """
        return f"{self.name} {self.embeddings}"

### Класс `Country` для таблицы `country`

Таблица `country` содержит следующие поля:

- `iso`: `Primary Key` - `ISO` двухсимвольный код страны;
- `iso_3`- `ISO` трехсимвольный код страны;
- `country` - страна;
- `capital` - столица страны;
- `area_in_sq_km` - площадь в кв. км;
- `population` - численость населения;
- `continent` - континент;
- `tld` - домен;
- `currency_code` - код валюты;
- `currency_name` - им валюты;
- `phone` - телефонный код;
- `languages` - языки государства.

In [26]:
class Country(Base):
    """
    Класс Country для хранения информации о странах.
    """
    # имя таблицы
    __tablename__ = "country"
    # комментарий с описанием таблицы
    __tableargs__ = {"comment": "Таблица со странами."}
    # задаем в переменные параметры столбцов в таблице БД, имя переменной является именем столбца
    iso = Column(
        String,
        nullable=False,
        unique=True,
        primary_key=True,
        comment="id, ISO 2-letter country code",
    )
    iso_3 = Column(String, comment="ISO 3-letter country code")
    country = Column(String, comment="country name")
    capital = Column(String, comment="country capital")
    area_in_sq_km = Column(Float, comment="country area")
    population = Column(String, comment="country population")
    continent = Column(String, comment="country continent")
    tld = Column(String, comment="country domen")
    currency_code = Column(String, comment="country currency code")
    currency_name = Column(String, comment="country currency name")
    phone = Column(String, comment="country phone code")
    languages = Column(String, comment="country base languages")

    def __repr__(self):
        """
        Метод __repr__.
        Возвращает строковое представление объекта.

        Возвращаемое значение:
            Строка (str): строковое представление объекта с названиями столбцов в таблице.
        """
        return f"{self.iso} {self.iso_3} {self.country} {self.capital} {self.area_in_sq_km} {self.population} {self.continent} {self.tld} {self.currency_code} {self.currency_name} {self.phone} {self.languages} "

### Класс `Admin` для таблицы `admincode`

Таблица `admincode` содержит следующие поля:

- `admin_code`: `Primary Key` - код области;
- `name` - имя области;
- `name_ascii` - имя области в `ascii` символах.

In [27]:
class Admin(Base):
    """
    Класс Admin для хранения информации об областях.
    """
    # имя таблицы
    __tablename__ = "admincode"
    # комментарий с описанием таблицы
    __tableargs__ = {"comment": "Таблица с областями."}

    # задаем в переменные параметры столбцов в таблице БД, имя переменной является именем столбца
    admin_code = Column(String, nullable=False, unique=True, primary_key=True)
    name = Column(String, comment="name of geographical point (utf8)")
    name_ascii = Column(
        String, comment="name of geographical point in plain ascii characters"
    )

    def __repr__(self):
        """
        Метод __repr__.
        Возвращает строковое представление объекта.

        Возвращаемое значение:
            Строка (str): строковое представление объекта с названиями столбцов в таблице.
        """
        return f"{self.admin_code} {self.name} {self.name_ascii}"

### Класс `City` для таблицы `city`

Таблица `city` содержит следующие поля:

- `city_geoname_id`: `Primary Key` - `geonameid` код города;
- `name` - имя города, `ForeignKey` ссылается на таблицу `embeddings` поле `name`,
- `asciiname` - имя города в `ascii` символах;
- `alternatenames` - альтернативные названия городов;
- `latitude` - широта в градусах;
- `longitude` - долгота в градусах;
- `feature_class` - класс административного деления http://www.geonames.org/export/codes.html;
- `feature_code` - код административного деления http://www.geonames.org/export/codes.html;
- `country_code_iso` - код страны, `ForeignKey` ссылается на таблицу `country` поле `iso`;
- `population` - население города в тыс. человек;
- `timezone` - временная зона города;
- `admin_code` код области города, `ForeignKey` ссылается на таблицу `admincode` поле `admin_code`.

In [28]:
class City(Base):
    """
    Класс City для хранения информации о городах.
    """
    # имя таблицы
    __tablename__ = "city"
    # комментарий с описанием таблицы
    __tableargs__ = {"comment": "Таблица с городами."}
    # задаем в переменные параметры столбцов в таблице БД, имя переменной является именем столбца
    city_geoname_id = Column(
        Integer,
        nullable=False,
        unique=True,
        primary_key=True,
        autoincrement=True,
        comment="id of record in geonames database",
    )
    name = Column(
        String,
        ForeignKey(
            "embeddings.name",
            name="fk_name",
            onupdate="CASCADE",
            ondelete="CASCADE",
        ),
        comment="name of geographical point (utf8)",
    )
    asciiname = Column(
        String, comment="name of geographical point in plain ascii characters"
    )
    alternatenames = Column(
        String,
        comment="alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table",
    )
    latitude = Column(Float, comment="latitude in decimal degrees")
    longitude = Column(Float, comment="longitude in decimal degrees")
    feature_class = Column(
        String, comment="see http://www.geonames.org/export/codes.html"
    )
    feature_code = Column(
        String, comment="see http://www.geonames.org/export/codes.html"
    )
    country_code_iso = Column(
        String,
        ForeignKey(
            "country.iso",
            name="fk_country_code_iso",
            onupdate="CASCADE",
            ondelete="CASCADE",
        ),
        comment="ISO-3166 2-letter country code, FK",
    )
    population = Column(Integer, comment="population")
    timezone = Column(String, comment="the iana timezone")
    admin_code = Column(
        String,
        ForeignKey(
            "admincode.admin_code",
            name="fk_admin_code",
            onupdate="CASCADE",
            ondelete="CASCADE",
        ),
        comment="code of admin division",
    )
    # задаем взаимосвязи между таблицами
    country = relationship("Country", backref="quote_country", lazy="subquery")
    admincode = relationship(
        "Admin", backref="quote_admincode", lazy="subquery")
    embeddings = relationship(
        "Vectors", backref="quote_embeddings", lazy="subquery")

    def __repr__(self):
        """
        Метод __repr__.
        Возвращает строковое представление объекта.

        Возвращаемое значение:
            Строка (str): строковое представление объекта с названиями столбцов в таблице.
        """
        return f"{self.city_geoname_id} {self.name} {self.asciiname} {self.alternatenames} {self.latitude} {self.longitude} {self.feature_class} {self.feature_code} {self.country_code_iso} {self.population} {self.timezone} {self.admin_code}"

**Создаем подключение к БД через `create_engine`**

In [29]:
engine = create_engine(CONN_STR_GEONAMES)

**Создаем таблицы БД вызовом метода `metadata.create_all` из `declarative_base()`**

In [30]:
Base.metadata.create_all(engine)

## Загрузка данных в таблицы `admincode`, `embeddings`, `country`, `city` в БД `geonames`

Для загрузки получения данных реализован класс `DataFrameSQL`.

Класс реализует следующие возможности:

- первичная загрузка данных в таблицы базы данных;
- добавление или замена данных, с возможностью снятия ограничения внешнего ключа и возврата ограничений обратно;
- получение данных из базы данных.

In [31]:
class DataFrameSQL:
    """
    Класс для сохранения данных из Pandas DataFrame в базу данных Postgres и наоборот.
    """

    def __init__(self, engine):
        """
        Инициализация объекта для работы с базой данных.

        Параметры:
            engine (sqlalchemy.engine): объект SQLAlchemy Engine для подключения к базе данных.
        """
        self.engine = engine

    def to_sql(
            self,
            df,
            table_name,
            if_exists="append",
            chunksize=10000,
            method="multi",
            index=False,
            dtype=None,
            fk_restriction=False,
    ):
        """
        Метод to_sql.
        Сохраняет DataFrame в базу данных.

         Параметры:
               df (pd.DataFrame): датафрейм, который нужно сохранить,
               table_name (str): наименование таблицы в базе данных, в которую нужно сохранить DataFrame,
               if_exists (bool): опция для действий при конфликте существующих записей, по умолчанию
                                 равно 'append',
               chunksize (int): количество строк для записи за один запрос к базе данных, по умолчанию
                                равно 10000,
               method (str): метод вставки данных в базу данных, по умолчанию равно 'multi',
               index (boll): опция для включения индекса в базу данных, по умолчанию равно False,
               dtype (dict): словарь для указания типов данных столбцов при сохранении в базу данных,
                             по умолчанию равно None,
               fk_restriction (bool): опция для управления ограничениями внешнего ключа при сохранении
                                      данных, по умолчанию равно False.
        """
        # если True
        if fk_restriction:
            # снятие ограничения внешнего ключа в зависимости от имени таблицы
            with self.engine.connect() as conn:
                if table_name == "embeddings":
                    conn.execute(
                        text("ALTER TABLE city DROP CONSTRAINT fk_name"))
                    conn.commit()
                elif table_name == "country":
                    conn.execute(
                        text("ALTER TABLE city DROP CONSTRAINT fk_country_code_iso")
                    )
                    conn.commit()
                else:
                    conn.execute(
                        text("ALTER TABLE city DROP CONSTRAINT fk_admin_code"))
                    conn.commit()
            conn.close()

        # загрузка данных из DataFrame в базу данных
        print(
            f"Загружаем датафрейм в таблицу {table_name} базы данных geonames ...")
        df.to_sql(
            table_name,
            con=self.engine,
            if_exists=if_exists,
            chunksize=chunksize,
            method=method,
            index=index,
            dtype=dtype,
        )
        print(f"Загружено {len(df)} записей!")
        # если True
        if fk_restriction:
            # восстановление первичного ключа и ограничений внешнего ключа после загрузки данных
            with self.engine.connect() as conn:
                if table_name == "embeddings":
                    conn.execute(
                        text("ALTER TABLE embeddings ADD PRIMARY KEY (name)"))
                    conn.commit()
                    conn.execute(
                        text(
                            "ALTER TABLE city ADD CONSTRAINT fk_name FOREIGN KEY (name) REFERENCES embeddings(name)"
                        )
                    )
                    conn.commit()
                elif table_name == "country":
                    conn.execute(
                        text("ALTER TABLE iso ADD PRIMARY KEY (country)"))
                    conn.commit()
                    conn.execute(
                        text(
                            "ALTER TABLE city ADD CONSTRAINT fk_country_code_iso FOREIGN KEY (country_code_iso) "
                            "REFERENCES country(iso)"
                        )
                    )
                    conn.commit()
                else:
                    conn.execute(
                        text("ALTER TABLE admincode ADD PRIMARY KEY (admin_code)")
                    )
                    conn.commit()
                    conn.execute(
                        text(
                            "ALTER TABLE city ADD CONSTRAINT fk_admin_code FOREIGN KEY (admin_code) REFERENCES "
                            "admincode(admin_code)"
                        )
                    )
                    conn.commit()
            conn.close()

    @staticmethod
    def check_country(countries):
        """
        Статический метод check_country класса DataFrameSQL.
        Метод проверяет тип введенной перемменой countries и
        преобразовывает значение переменной в строку для формирования запроса к БД.
        Например, сначала список ["Russia", "Kazakhstan"] преобразовывается в вид
        ["'Russia'", "'Kazakhstan'"], т.е. добавляются кавычки и затем список преобразовывается в строку.
        Это необходимо для передачи в конструкцию WHERE ... IN () ... SQL запроса.
         Параметры:
               countries(str): страна или список стран.
         Возвращаемое значение:
               countries (str): переработанная переменная для запроса query.
         """
        # проверка на соответствие переменной countries на тип str или list
        if not isinstance(countries, (str, list)):
            raise TypeError(
                f"Не соответствует тип переменной countries, должен быть тип str или list. Датасет не будет создан."
            )
        # проверка переменной countries на ненулевую длину
        if len(countries) == 0:
            raise ValueError(
                f"Пустая строка или пустой список вместо countries. . Датасет не будет создан."
            )
        # если на вход подается строка с одной страной, то добавляются кавычки и возвращается измененная строка
        if isinstance(countries, str):
            countries = "'" + countries + "'"
            return countries
        # если на вход подается список с одной или несколькими странами, то добавляются кавычки к каждому элементу и
        # возвращается измененная строка
        else:
            countries = ["'" + country + "'" for country in countries]
            return ", ".join(countries)

    def from_sql(self, query=None, countries=None, population=15000):
        """
        Метод from_sql.
        Загружает данные из БД Postgres в DataFrame Pandas.

         Параметры:
               query (str): SQL запрос к БД, по умолчанию равно None,
               countries (str or list): страна или список стран для ограничения в запросе по странам,
                                        по умолчанию равно None,
               population (int): население в городах, по умолчанию равно 15000.
         """
        # преобразование стран через вызов статического метода check_country
        countries = DataFrameSQL.check_country(countries)
        # запрос к БД
        # т.к. в запросе есть условие WHERE co.country IN ({}) AND ci.population >= {}
        # необходимо добавить format(countries, population), чтобы страны и население отобразились в запросе
        query = query.format(
            countries, population
        )
        # метод пандас для получения датасета из БД
        dataset = pd.read_sql(query, con=self.engine)
        # возвращаемый датасет
        return dataset

**Функция `addapt_numpy_float32`**

Эта функция нужна для загрузки массива эмбеддингов в таблицу `embeddings`.

Этот код связан с библиотекой `psycopg2`, которая предоставляет адаптеры для преобразования объектов `Python` в типы данных, понятные для `PostgreSQL` при выполнении `SQL`-запросов.

Без этого адаптера не получится загрузить эмбеддинги в базу данных.

1. `addapt_numpy_float32(numpy_float32)` - это определение функции `addapt_numpy_float32`, которая принимает объект типа `numpy.float32`. В данном случае функция просто возвращает этот объект. В библиотеке `psycopg2`, `AsIs` представляет собой класс-обертку, который позволяет передавать значения напрямую в `SQL`-запросы, обойдя процесс преобразования и экранирования, который обычно выполняется при выполнении `SQL`-запросов. `AsIs` используется для предотвращения автоматического экранирования и преобразования значений при встраивании их в `SQL`-запросы. Например, если вы хотите передать значение как есть, без изменений или экранирования, можно обернуть его в `AsIs`, чтобы показать, что оно должно быть вставлено в `SQL`-запрос без изменений. Полезно использовать `AsIs`, когда есть уверенность, что значение безопасно для вставки в запрос, например, когда значение уже прошло проверку безопасности или когда оно представляет собой некоторое сложное `SQL`-выражение. Таким образом, `AsIs` указывает драйверу `PostgreSQL` обрабатывать переданное значение "как есть", без дополнительных преобразований или экранирования.

2. `register_adapter(np.float32, addapt_numpy_float32)` - этот код регистрирует адаптер для типа данных `numpy.float32`. Он связывает указанный тип данных с соответствующей функцией преобразования. Суть в том, что когда объект типа `numpy.float32` будет встречен при передаче данных в `PostgreSQL`, будет вызвана функция `addapt_numpy_float32`, чтобы преобразовать этот объект в формат, понятный для `PostgreSQL`.

В данном случае есть массив `numpy` типа `np.float32`, и надо передать этот массив в `PostgreSQL`. `PostgreSQL` может требовать преобразования типов, и это именно то, чем занимается этот кусок кода. Он позволяет адаптировать тип `np.float32` для корректного сохранения в базе данных `PostgreSQL`.

In [32]:
def addapt_numpy_float32(numpy_float32):
    """
    Функция адаптер типа np.float32.
     Параметры:
           принимает объект типа numpy.float32
     Возвращаемое значение:
           numpy.float32 через  класс-обертку AsIs
    """
    return AsIs(numpy_float32)


# применение register_adapter
register_adapter(np.float32, addapt_numpy_float32)

**Создаем экземпляр класса `DataFrameSQL`**

In [33]:
# инициализируем класс с уже существующим соединением с базой данных
data_sql = DataFrameSQL(engine)

**Загружаем данные в таблицы методом `to_sql` класса `DataFrameSQL`**

Так как таблица `city` содержит внешние ключи, её загружаем последней.

In [34]:
# cохраняем данные в таблицу 'admincode'
data_sql.to_sql(admin_codes, "admincode")

# cохраняем данные в таблицу 'embeddings' с использованием параметра dtype для столбца 'embeddings'
data_sql.to_sql(embeddings, "embeddings", dtype={"embeddings": ARRAY(REAL)})

# cохраняем данные в таблицу 'country'
data_sql.to_sql(countries, "country")

# cохраняем данные в таблицу 'city'
data_sql.to_sql(cities, "city")

Загружаем датафрейм в таблицу admincode базы данных geonames ...
Загружено 3921 записей!
Загружаем датафрейм в таблицу embeddings базы данных geonames ...
Загружено 169300 записей!
Загружаем датафрейм в таблицу country базы данных geonames ...
Загружено 252 записей!
Загружаем датафрейм в таблицу city базы данных geonames ...
Загружено 200593 записей!


**Очистка памяти**

In [35]:
del cities, countries, admin_codes, embeddings
gc.collect()

194

## Получение данных из БД `geonames`

**Загружаем данные из базы данных методом `from_sql` класса `DataFrameSQL`**

Для загрузки используются ранее объявленные переменные:

- `QUERY` - `SQL`-запрос, 
- `COUNTRIES_LST` - список стран, 
- `POPULATION` - чистенность населения.

In [36]:
data = data_sql.from_sql(query=QUERY, countries=COUNTRIES_LST, population=POPULATION)

**Информация по датасету**

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174 entries, 0 to 1173
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   geoname_id      1174 non-null   int64  
 1   name            1174 non-null   object 
 2   alternatenames  1174 non-null   object 
 3   oblast          1174 non-null   object 
 4   country         1174 non-null   object 
 5   capital         1174 non-null   object 
 6   currency_name   1174 non-null   object 
 7   timezone        1174 non-null   object 
 8   latitude        1174 non-null   float64
 9   longitude       1174 non-null   float64
 10  embeddings      1174 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 101.0+ KB


## Поиск городов

Для финального поиска реализован класс `FindCity`.

Класс имеет следующие возможности:

- коррекцию опечаток,
- расширенную коррекцию опечаток или сокращений при помощи поска по аьтернативным названиям городов,
- возможность выбора вывода информации в виде таблицы или словаря,
- возможносьть сохранения словаря в `JSON` файл.

In [38]:
class FindCity:
    """
    Класс FindCity для поиска города по векторному представлению.
    """

    def __init__(self, model_id=None, device="cpu", dataset=None, emb_col=None, cols_output=None):
        """
        Инициализация объекта класса FindCity для поиска города.

         Параметры::
            model_id (str): имя модели для векторизации, по умолчанию равно None,
            device (str): акселератор для создания векторов CPU или GPU, по
                          умолчанию равно 'cpu',
            dataset (pd.DataFrame): датасет, содержащий векторные представления городов,
                                    по которому будет осуществляться поиск, по
                                    умолчанию равно None,
            emb_col (str): наименование столбца с векторами представлений, по
                           умолчанию равно None,
            cols_output (list): список наименований столбцов для вывода результата, по
                                умолчанию равно None.
        """
        self.model_id = model_id
        self.device = device
        self.dataset = dataset
        self.emb_col = emb_col
        self.cities_emb = np.array(
            list(self.dataset[self.emb_col]), dtype=np.float32)
        self.model = SentenceTransformer(self.model_id, device=self.device)
        self.cols_output = cols_output

    @staticmethod
    def spell_checker(city=None):
        """
        Статический метод spell_checker класса FindCity.
        Проверка названия города для исправления возможных опечаток.
        Метод для первичной проверки опечаток, использует метод для
        проверки правописания из пакета yaspeller (Яндекс Спеллер)

         Параметры:
             city (str): название города для проверки, по умолчанию равно None.

         Возвращаемое значение:
             city (str): скорректированное название города или исходное значение,
                         в случае невозможности корректировки.
         """
        # в переменную res записывается True или False.
        # вызывается метод check из yaspeller, если нету ошибок res = True,
        # в противном случае res = False
        res = check(city, lang="ru")
        # если True
        if res.is_ok:
            # возврат исходного слова.
            return city
        # если False
        else:
            # с помощью метода first_match находим первое близкое правильное значение.
            city = res.first_match()
            # возврат исправленного названия города.
            return city

    @staticmethod
    def advanced_spell_checker(city=None, dataset=None):
        """
        Статический метод advanced_spell_checker класса FindCity.
        Расширенная проверка названия города в случае невозможности исправить
        первым методом spell_checker.
        Метод ищет совпадения заданного города в альтернативных именах. Если
        совпадение найдено, возвращается имя города из поля name, если нет,
        то возвращается исходное значение. В процессе используются методы:
         - translit библиотеки transliterate,
         - process библиотеки fuzzywuzzy.
        В основном расширенная проверка нужна для сокращений:
         - МСК - Москва,
         - СПБ - Санкт-Петербург.
         Параметры:
              city (str): название города для проверки, по умолчанию равно None.
              dataset (pd.DataFrame): датасет с городами и альтернативными именами городов,
                                      по умолчанию равно None.
         Возвращаемое значение:
              city (str): скорректированное название города или исходное значение,
                        в случае невозможности корректировки.
        """
        # создаем словарь из датасета, где ключ это значение из поля name, а значения
        # это строка альтернативных имен.
        cities_dict = (
            dataset.groupby("name")["alternatenames"]
            .apply(lambda x: ",".join(map(str, x)))
            .to_dict()
        )
        # транслитерация введенного на русском языке названия города
        city_t = translit(city, "ru", reversed=True)
        # пустой список для найденных городов
        key_lst = []
        # цикл по словарю
        for key, value in cities_dict.items():
            # проверка наличия имени города в исходном или транслитном значении в разбитой на список строке
            # c альтернативными названиями
            if (city.lower() in value.lower().split(", ")) or (
                    city_t.lower() in value.lower().split(", ")
            ):
                # если поиск успешен добавляем ключ - имя города в список
                key_lst.append(key)
            # если поиск безрезультатен
            else:
                # проходим циклом по списку из разделенной строки с альтернативными названиями
                for item in value.lower().split(", "):
                    # транслитерация названия
                    item_t = translit(item, "ru", reversed=True)
                    # альтернативное имя начинается с искомого слова в оригинале или транслите
                    if item.startswith(city) or item_t.startswith(city_t):
                        # то добавляем ключ в список имен
                        key_lst.append(key)
        # в списке может быть много городов и дублей городов, для этого через множество удалим дубли
        key_lst = list(set(key_lst))
        # если длина списка 1 значение
        if len(key_lst) == 1:
            # то в возвращаемую переменную присваиваем 0-й элемент списка
            city = key_lst[0]
            # возвращаемое значение
            return city
        # если городов в списке больше двух
        elif len(key_lst) > 1:
            # то методом process.extract библиотеки fuzzywuzzy вытаскиваем наиболее близкий по расстоянию Левенштейна
            word = process.extractOne(city_t, key_lst)
            # т.к. в word записан список кортежей, то берем из него только название города
            city = word[0][0]
            # возвращаемое значение
            return city
        # если ничего не найдено
        else:
            city = translit(city.lower(), "ru", reversed=True)
            # то возвращаем транслитное значение введенного слова
            return city

    def get_city(
            self,
            city=None,
            top_k=1,
            adv_spell_check=False,
            output_dict_json=False,
            save_json_file=False,
            work_dir=None,
    ):
        """
        Получение информации о городе на основе введенного названия.

         Параметры:
            city (str): название города для поиска, по умолчанию равно None,
            top_k (int): количество наиболее похожих городов для вывода, по умолчанию равно 1,
            adv_spell_check (bool): флаг использования расширенной проверки названия города,
                                    по умолчанию равно False,
            output_dict_json (bool): флаг вывода результата в формате JSON, по умолчанию равно False,
            save_json_file (bool): флаг сохранения результата в JSON-файл, по умолчанию равно False,
            work_dir (str): каталог для сохранения JSON-файла, по умолчанию равно None/

         Возвращаемое значение:
            Отсутствует. Ввод исключительно на экран или сохранение в файл JSON.
            Выводится либо pd.DataFrame с информацией о городе и сходстве с искомым названием,
            либо словарь с этой же информацией.
        """
        # первичная проверка на исправление ошибок
        city = FindCity.spell_checker(city=city)
        # если True
        if adv_spell_check:
            # запускаем расширенную проверку опечаток или сокращений
            city = FindCity.advanced_spell_checker(
                city=city, dataset=self.dataset)
        # поучаем вектор имени города
        full_city_vector = self.model.encode([city], device=self.device)
        # выбираем количество схожих городов для вывода
        tops = min(top_k, len(self.cities_emb))
        # получаем результат при помощи метода util.semantic_search, где по дефолту косинусное сходство
        score = util.semantic_search(
            full_city_vector, self.cities_emb, top_k=tops)[0]
        # список индексов имен городов из датасета
        lst_idx = [score[i]["corpus_id"] for i in range(len(score))]
        # список с косинусным сходством по индексу
        scores = [score[i]["score"] for i in range(len(score))]
        # формируем результирующий датасет из входного по отобранным индексам
        result_df = self.dataset[self.cols_output].iloc[lst_idx]
        # добавляем колонку со скорингом
        result_df["cos_sim_score"] = scores
        # если нужен вывод в виде словаря
        if output_dict_json:
            # формируем словарь из датафрейма
            output_dict = result_df.to_dict(orient="records")
            # и красиво его выводим
            pprint(output_dict, sort_dicts=False)
            # если нужно – то сохраняем json файл
            if save_json_file:
                with open(os.path.join(work_dir, f"{city}.json"), "w") as fp:
                    json.dump(output_dict, fp)
        # иначе
        else:
            # выводим на экран датафрейм
            display(result_df)

**Создаем экземпляр класса `FindCity`**

Для этого передаём параметры для инициализации класса.
Часть переменных описана в начале блокнота. 

In [39]:
%%time
finder = FindCity(model_id=MODEL_ID, device="cpu", dataset=data,
                  emb_col="embeddings", cols_output=COLS_OUTPUT)

CPU times: user 5.45 s, sys: 2.7 s, total: 8.15 s
Wall time: 10.6 s


### Проверка работоспособности метода `get_city` класса `FindCity` 

In [40]:
%%time
finder.get_city(city="мысква", top_k=3)

,geoname_id,name,oblast,country,capital,currency_name,timezone,latitude,longitude,cos_sim_score
567,524901,Moscow,Moscow,Russia,Moscow,Ruble,Europe/Moscow,55.75000,37.62500,0.902249
568,857690,Moskovskiy,Moscow,Russia,Moscow,Ruble,Europe/Moscow,55.59375,37.34375,0.790183
571,524712,Mozhaysk,Moscow Oblast,Russia,Moscow,Ruble,Europe/Moscow,55.50000,36.03125,0.692705


CPU times: user 224 ms, sys: 40.2 ms, total: 265 ms
Wall time: 495 ms


Как видим опечатки исправлены и на первом месте `Москва`.

In [41]:
%%time
finder.get_city(city="Ростов", top_k=3)

,geoname_id,name,oblast,country,capital,currency_name,timezone,latitude,longitude,cos_sim_score
784,501183,Rostov,Yaroslavl Oblast,Russia,Moscow,Ruble,Europe/Moscow,57.18750,39.40625,0.917979
785,501175,Rostov-na-Donu,Rostov,Russia,Moscow,Ruble,Europe/Moscow,47.21875,39.71875,0.803299
788,1493467,Rubtsovsk,Altai Krai,Russia,Moscow,Ruble,Asia/Barnaul,51.50000,81.18750,0.598959


CPU times: user 297 ms, sys: 9 µs, total: 297 ms
Wall time: 302 ms


Найдены оба `Ростова`.

In [42]:
%%time
finder.get_city(city="Новгород", top_k=3)

,geoname_id,name,oblast,country,capital,currency_name,timezone,latitude,longitude,cos_sim_score
616,520555,Nizhniy Novgorod,Nizhny Novgorod Oblast,Russia,Moscow,Ruble,Europe/Moscow,56.34375,44.000000,0.746459
1042,519336,Velikiy Novgorod,Novgorod Oblast,Russia,Moscow,Ruble,Europe/Moscow,58.53125,31.265625,0.733398
630,518909,Novodvinsk,Arkhangelskaya,Russia,Moscow,Ruble,Europe/Moscow,64.43750,40.812500,0.655147


CPU times: user 206 ms, sys: 4.01 ms, total: 210 ms
Wall time: 239 ms


И оба `Новгорода`.

In [43]:
%%time
finder.get_city(city="Екатерин", top_k=3)

,geoname_id,name,oblast,country,capital,currency_name,timezone,latitude,longitude,cos_sim_score
1105,1486209,Yekaterinburg,Sverdlovsk Oblast,Russia,Moscow,Ruble,Asia/Yekaterinburg,56.84375,60.62500,0.817051
280,2023469,Irkutsk,Irkutsk Oblast,Russia,Moscow,Ruble,Asia/Irkutsk,52.31250,104.31250,0.699812
1103,468307,Yegorlykskaya,Rostov,Russia,Moscow,Ruble,Europe/Moscow,46.56250,40.65625,0.655866


CPU times: user 212 ms, sys: 66 µs, total: 212 ms
Wall time: 250 ms


А тут нашли `Екатеринбург`

In [44]:
%%time
finder.get_city(city="Владиво", top_k=3)

,geoname_id,name,oblast,country,capital,currency_name,timezone,latitude,longitude,cos_sim_score
1057,2013348,Vladivostok,Primorye,Russia,Moscow,Ruble,Asia/Vladivostok,43.09375,131.87500,0.901027
1055,473249,Vladikavkaz,North Ossetia–Alania,Russia,Moscow,Ruble,Europe/Moscow,43.03125,44.65625,0.749308
1074,471457,Vostryakovo,Moscow Oblast,Russia,Moscow,Ruble,Europe/Moscow,55.40625,37.81250,0.716811


CPU times: user 338 ms, sys: 0 ns, total: 338 ms
Wall time: 424 ms


Нашли `Владивосток` и похожий по названию `Владикавказ`.

In [45]:
%%time
finder.get_city(city="спб", top_k=3)

,geoname_id,name,oblast,country,capital,currency_name,timezone,latitude,longitude,cos_sim_score
227,609924,Embi,Aqtöbe,Kazakhstan,Nur-Sultan,Tenge,Asia/Aqtobe,48.84375,58.15625,0.490213
664,1496421,Ob’,Novosibirsk Oblast,Russia,Moscow,Ruble,Asia/Novosibirsk,55.00000,82.68750,0.479240
874,493160,Sibay,Bashkortostan Republic,Russia,Moscow,Ruble,Asia/Yekaterinburg,52.71875,58.65625,0.475520


CPU times: user 270 ms, sys: 10.8 ms, total: 281 ms
Wall time: 271 ms


А вот тут по скоращению `спб` нужного результата нету.

Стоит попробовать расширенный корректор `adv_spell_check=True`.

In [46]:
%%time
finder.get_city(city="спб", top_k=3, adv_spell_check=True)

,geoname_id,name,oblast,country,capital,currency_name,timezone,latitude,longitude,cos_sim_score
800,498817,Saint Petersburg,St.-Petersburg,Russia,Moscow,Ruble,Europe/Moscow,59.93750,30.3125,1.000000
721,1520172,Petropavl,North Kazakhstan,Kazakhstan,Nur-Sultan,Tenge,Asia/Almaty,54.87500,69.1250,0.607880
720,8504948,Petrogradka,St.-Petersburg,Russia,Moscow,Ruble,Europe/Moscow,59.96875,30.3125,0.599224


CPU times: user 764 ms, sys: 0 ns, total: 764 ms
Wall time: 849 ms


Результат получен

In [47]:
%%time
finder.get_city(city="С.питербург", top_k=3)

,geoname_id,name,oblast,country,capital,currency_name,timezone,latitude,longitude,cos_sim_score
800,498817,Saint Petersburg,St.-Petersburg,Russia,Moscow,Ruble,Europe/Moscow,59.93750,30.31250,0.755859
720,8504948,Petrogradka,St.-Petersburg,Russia,Moscow,Ruble,Europe/Moscow,59.96875,30.31250,0.645943
718,510291,Peterhof,St.-Petersburg,Russia,Moscow,Ruble,Europe/Moscow,59.87500,29.90625,0.636605


CPU times: user 297 ms, sys: 3.81 ms, total: 300 ms
Wall time: 341 ms


Аналогично

In [48]:
%%time
finder.get_city(city="остона", top_k=3)

,geoname_id,name,oblast,country,capital,currency_name,timezone,latitude,longitude,cos_sim_score
68,1526273,Astana,Astana,Kazakhstan,Nur-Sultan,Tenge,Asia/Almaty,51.18750,71.437500,0.695810
285,555746,Istra,Moscow Oblast,Russia,Moscow,Ruble,Europe/Moscow,55.90625,36.875000,0.579331
691,514171,Ostrov,Pskov Oblast,Russia,Moscow,Ruble,Europe/Moscow,57.34375,28.359375,0.571323


CPU times: user 340 ms, sys: 0 ns, total: 340 ms
Wall time: 374 ms


И здесь ошибки исправлены.

In [49]:
%%time
finder.get_city(city="Благавещинск", top_k=3)

,geoname_id,name,oblast,country,capital,currency_name,timezone,latitude,longitude,cos_sim_score
127,2026609,Blagoveshchensk,Amur Oblast,Russia,Moscow,Ruble,Asia/Yakutsk,50.28125,127.56250,0.871622
126,576116,Blagoveshchensk,Bashkortostan Republic,Russia,Moscow,Ruble,Asia/Yekaterinburg,55.03125,55.96875,0.871622
106,577893,Belorechensk,Krasnodar Krai,Russia,Moscow,Ruble,Europe/Moscow,44.78125,39.87500,0.753271


CPU times: user 286 ms, sys: 0 ns, total: 286 ms
Wall time: 303 ms


И тут исправлены опечатки.

А вот так выглядит вывод в виде словаря:

In [50]:
%%time
finder.get_city(city="Благавещинск", top_k=3, output_dict_json=True)

[{'geoname_id': 2026609,
  'name': 'Blagoveshchensk',
  'oblast': 'Amur Oblast',
  'country': 'Russia',
  'capital': 'Moscow',
  'currency_name': 'Ruble',
  'timezone': 'Asia/Yakutsk',
  'latitude': 50.28125,
  'longitude': 127.5625,
  'cos_sim_score': 0.8716219663619995},
 {'geoname_id': 576116,
  'name': 'Blagoveshchensk',
  'oblast': 'Bashkortostan Republic',
  'country': 'Russia',
  'capital': 'Moscow',
  'currency_name': 'Ruble',
  'timezone': 'Asia/Yekaterinburg',
  'latitude': 55.03125,
  'longitude': 55.96875,
  'cos_sim_score': 0.8716219663619995},
 {'geoname_id': 577893,
  'name': 'Belorechensk',
  'oblast': 'Krasnodar Krai',
  'country': 'Russia',
  'capital': 'Moscow',
  'currency_name': 'Ruble',
  'timezone': 'Europe/Moscow',
  'latitude': 44.78125,
  'longitude': 39.875,
  'cos_sim_score': 0.7532709240913391}]
CPU times: user 206 ms, sys: 7.98 ms, total: 214 ms
Wall time: 297 ms


## Вывод

В результате работы над проектом была разработана работоспособная модель для подбора наиболее подходящих названий с geonames. Например Ереван -> Yerevan.

- на примере РФ и Казахстана для городов с населением от 15000 человек протестирована работа модели,
- реализован формат данных на выходе: таблица или список словарей,
- есть возможность выбора расширенного корректора опечаток,
- есть возможно выбора вывода количества городов,
- хранение в `PostgreSQL` данных geonames,
- хранение векторизованных промежуточных данных в `PostgreSQL`,
- предусмотрены методы для настройки подключения к `БД`,
- предусмотреть методы для добавления векторов новых гео названий.

Проет реализован от создания базы данный и до вывода конечного результата.

Модель можно рекомендовать заказчику для дальнейшей доработки и реализации.